In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4040')

sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/02 02:31:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pybloom import BloomFilter, ScalableBloomFilter
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType

In [3]:
# Read the CSV files
df = spark.read.option("header", "true").csv("../shared/midterm/drive_stats_2019_Q1/*.csv")

df.printSchema()
df.head()

root
 |-- date: string (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: string (nullable = true)
 |-- failure: string (nullable = true)
 |-- smart_1_normalized: string (nullable = true)
 |-- smart_1_raw: string (nullable = true)
 |-- smart_2_normalized: string (nullable = true)
 |-- smart_2_raw: string (nullable = true)
 |-- smart_3_normalized: string (nullable = true)
 |-- smart_3_raw: string (nullable = true)
 |-- smart_4_normalized: string (nullable = true)
 |-- smart_4_raw: string (nullable = true)
 |-- smart_5_normalized: string (nullable = true)
 |-- smart_5_raw: string (nullable = true)
 |-- smart_7_normalized: string (nullable = true)
 |-- smart_7_raw: string (nullable = true)
 |-- smart_8_normalized: string (nullable = true)
 |-- smart_8_raw: string (nullable = true)
 |-- smart_9_normalized: string (nullable = true)
 |-- smart_9_raw: string (nullable = true)
 |-- smart_10_normalized: string (nullable = tru

Row(date='2019-03-05', serial_number='Z305B2QN', model='ST4000DM000', capacity_bytes='4000787030016', failure='0', smart_1_normalized='117', smart_1_raw='166138696', smart_2_normalized=None, smart_2_raw=None, smart_3_normalized='91', smart_3_raw='0', smart_4_normalized='100', smart_4_raw='12', smart_5_normalized='100', smart_5_raw='0', smart_7_normalized='85', smart_7_raw='374504040', smart_8_normalized=None, smart_8_raw=None, smart_9_normalized='68', smart_9_raw='28210', smart_10_normalized='100', smart_10_raw='0', smart_11_normalized=None, smart_11_raw=None, smart_12_normalized='100', smart_12_raw='12', smart_13_normalized=None, smart_13_raw=None, smart_15_normalized=None, smart_15_raw=None, smart_16_normalized=None, smart_16_raw=None, smart_17_normalized=None, smart_17_raw=None, smart_22_normalized=None, smart_22_raw=None, smart_23_normalized=None, smart_23_raw=None, smart_24_normalized=None, smart_24_raw=None, smart_168_normalized=None, smart_168_raw=None, smart_170_normalized=None

In [4]:
df.count()

9577046

In [5]:
df.show(5)

+----------+--------------+--------------------+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-

In [6]:
df = df.withColumn('date', df['date'].cast('date'))

df.printSchema()
df.head()

root
 |-- date: date (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: string (nullable = true)
 |-- failure: string (nullable = true)
 |-- smart_1_normalized: string (nullable = true)
 |-- smart_1_raw: string (nullable = true)
 |-- smart_2_normalized: string (nullable = true)
 |-- smart_2_raw: string (nullable = true)
 |-- smart_3_normalized: string (nullable = true)
 |-- smart_3_raw: string (nullable = true)
 |-- smart_4_normalized: string (nullable = true)
 |-- smart_4_raw: string (nullable = true)
 |-- smart_5_normalized: string (nullable = true)
 |-- smart_5_raw: string (nullable = true)
 |-- smart_7_normalized: string (nullable = true)
 |-- smart_7_raw: string (nullable = true)
 |-- smart_8_normalized: string (nullable = true)
 |-- smart_8_raw: string (nullable = true)
 |-- smart_9_normalized: string (nullable = true)
 |-- smart_9_raw: string (nullable = true)
 |-- smart_10_normalized: string (nullable = true)

Row(date=datetime.date(2019, 3, 5), serial_number='Z305B2QN', model='ST4000DM000', capacity_bytes='4000787030016', failure='0', smart_1_normalized='117', smart_1_raw='166138696', smart_2_normalized=None, smart_2_raw=None, smart_3_normalized='91', smart_3_raw='0', smart_4_normalized='100', smart_4_raw='12', smart_5_normalized='100', smart_5_raw='0', smart_7_normalized='85', smart_7_raw='374504040', smart_8_normalized=None, smart_8_raw=None, smart_9_normalized='68', smart_9_raw='28210', smart_10_normalized='100', smart_10_raw='0', smart_11_normalized=None, smart_11_raw=None, smart_12_normalized='100', smart_12_raw='12', smart_13_normalized=None, smart_13_raw=None, smart_15_normalized=None, smart_15_raw=None, smart_16_normalized=None, smart_16_raw=None, smart_17_normalized=None, smart_17_raw=None, smart_22_normalized=None, smart_22_raw=None, smart_23_normalized=None, smart_23_raw=None, smart_24_normalized=None, smart_24_raw=None, smart_168_normalized=None, smart_168_raw=None, smart_170_no

In [7]:
df.count()

9577046

In [8]:
df.show(5)

+----------+--------------+--------------------+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-

In [9]:
# Split the dataset by month: January, February and March

df_jan = df.filter(month(df['date']) == 1)
df_feb = df.filter(month(df['date']) == 2)
df_mar = df.filter(month(df['date']) == 3)

In [10]:
df_jan.printSchema()

root
 |-- date: date (nullable = true)
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- capacity_bytes: string (nullable = true)
 |-- failure: string (nullable = true)
 |-- smart_1_normalized: string (nullable = true)
 |-- smart_1_raw: string (nullable = true)
 |-- smart_2_normalized: string (nullable = true)
 |-- smart_2_raw: string (nullable = true)
 |-- smart_3_normalized: string (nullable = true)
 |-- smart_3_raw: string (nullable = true)
 |-- smart_4_normalized: string (nullable = true)
 |-- smart_4_raw: string (nullable = true)
 |-- smart_5_normalized: string (nullable = true)
 |-- smart_5_raw: string (nullable = true)
 |-- smart_7_normalized: string (nullable = true)
 |-- smart_7_raw: string (nullable = true)
 |-- smart_8_normalized: string (nullable = true)
 |-- smart_8_raw: string (nullable = true)
 |-- smart_9_normalized: string (nullable = true)
 |-- smart_9_raw: string (nullable = true)
 |-- smart_10_normalized: string (nullable = true)

In [11]:
df_jan.count() + df_feb.count() + df_mar.count()

9577046

In [12]:
# Display the total actual hard drives active during each month

# assuming that the drive cannot be brought back once it goes down, failure = 1, dead forever
# ignoring the drives which go dead at any point

df_jan_status = df_jan.groupBy(col("serial_number"), col("model"), col("smart_1_raw")).agg(sum("failure").alias("failure"))
df_jan_active = df_jan_status.filter((col("failure") == 0) & (col("smart_1_raw") == 0))

print("Total actual hard drives active during Jan: ", df_jan_active.count())
df_jan_active.show(5)

Total actual hard drives active during Jan:  28937


+--------------+--------------------+-----------+-------+
| serial_number|               model|smart_1_raw|failure|
+--------------+--------------------+-----------+-------+
|PL1331LAGBYYVH|HGST HMS5C4040ALE640|          0|    0.0|
|     57HBWEE5T|  TOSHIBA MQ01ABF050|          0|    0.0|
|PL1331LAGM2UEH|HGST HMS5C4040BLE640|          0|    0.0|
|PL2331LAHD7U9J|HGST HMS5C4040BLE640|          0|    0.0|
|PL2331LAHAZURJ|HGST HMS5C4040BLE640|          0|    0.0|
+--------------+--------------------+-----------+-------+
only showing top 5 rows



In [13]:
df_feb_status = df_feb.groupBy(col("serial_number"), col("model"), col("smart_1_raw")).agg(sum("failure").alias("failure"))
df_feb_active = df_feb_status.filter((col("failure") == 0) & (col("smart_1_raw") == 0))

print("Total actual hard drives active during Feb: ", df_feb_active.count())
df_feb_active.show(5)

Total actual hard drives active during Feb:  26061


+--------------+--------------------+-----------+-------+
| serial_number|               model|smart_1_raw|failure|
+--------------+--------------------+-----------+-------+
|PL1331LAGBYYVH|HGST HMS5C4040ALE640|          0|    0.0|
|     57HBWEE5T|  TOSHIBA MQ01ABF050|          0|    0.0|
|PL2331LAHD7U9J|HGST HMS5C4040BLE640|          0|    0.0|
|PL2331LAHAZURJ|HGST HMS5C4040BLE640|          0|    0.0|
|PL2331LAHBALNJ|HGST HMS5C4040BLE640|          0|    0.0|
+--------------+--------------------+-----------+-------+
only showing top 5 rows



In [14]:
df_mar_status = df_mar.groupBy(col("serial_number"), col("model"), col("smart_1_raw")).agg(sum("failure").alias("failure"))
df_mar_active = df_mar_status.filter((col("failure") == 0) & (col("smart_1_raw") == 0))

print("Total actual hard drives active during Mar: ", df_mar_active.count())
df_mar_active.show(5)

Total actual hard drives active during Mar:  27995


+--------------+--------------------+-----------+-------+
| serial_number|               model|smart_1_raw|failure|
+--------------+--------------------+-----------+-------+
|      ZA189JSN|        ST8000NM0055|          0|    0.0|
|PL1331LAGBYYVH|HGST HMS5C4040ALE640|          0|    0.0|
|     57HBWEE5T|  TOSHIBA MQ01ABF050|          0|    0.0|
|PL2331LAHD7U9J|HGST HMS5C4040BLE640|          0|    0.0|
|PL2331LAHAZURJ|HGST HMS5C4040BLE640|          0|    0.0|
+--------------+--------------------+-----------+-------+
only showing top 5 rows



In [15]:
df_jan_active.count() + df_feb_active.count(), df_mar_active.count()

(54998, 27995)

In [16]:
# Define the membership set as those hard drives (model name) with zero error rate 
# during January and February. Train a bloom filter with that set

sbf = ScalableBloomFilter(mode=ScalableBloomFilter.SMALL_SET_GROWTH)

In [17]:
df_jan_feb_active = df_jan_active.union(df_feb_active)

In [18]:
train_set = [sbf.add(row["model"]) for row in df_jan_feb_active.rdd.collect()]
train_set[:25]

[False,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True]

In [19]:
len(train_set)

54998

In [20]:
all([row["model"] in sbf for row in df_jan_feb_active.rdd.collect()])

True

In [21]:
# Test the March drives for set membership

test_set = [(row["model"] in sbf) for row in df_mar_active.rdd.collect()]
test_set[:25]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True]

In [22]:
len(test_set)

27995

In [23]:
@udf(returnType=BooleanType())
def is_member_set(model):
    return model in sbf

In [24]:
df_mar_result = df_mar_active.withColumn("is_member_set", is_member_set(col("model")))

df_mar_result.printSchema()
df_mar_result.count()

root
 |-- serial_number: string (nullable = true)
 |-- model: string (nullable = true)
 |-- smart_1_raw: string (nullable = true)
 |-- failure: double (nullable = true)
 |-- is_member_set: boolean (nullable = true)



27995

In [25]:
df_mar_result.show(5)

+--------------+--------------------+-----------+-------+-------------+
| serial_number|               model|smart_1_raw|failure|is_member_set|
+--------------+--------------------+-----------+-------+-------------+
|      ZA189JSN|        ST8000NM0055|          0|    0.0|         true|
|PL1331LAGBYYVH|HGST HMS5C4040ALE640|          0|    0.0|         true|
|     57HBWEE5T|  TOSHIBA MQ01ABF050|          0|    0.0|         true|
|PL2331LAHD7U9J|HGST HMS5C4040BLE640|          0|    0.0|         true|
|PL2331LAHAZURJ|HGST HMS5C4040BLE640|          0|    0.0|         true|
+--------------+--------------------+-----------+-------+-------------+
only showing top 5 rows



In [26]:
df_mar_result.filter(col("is_member_set") == True).count()

27473